### Monte Carlo: Black-Scholes-Merton

In [31]:
# Importando bibliotecas necessárias:
import numpy as np
import pandas as pd
import pandas_datareader.data as wb
from scipy.stats import norm

In [32]:
# Valor a receber com o execício da opção:
def d1(S, K, r, stdev, T):
    return (np.log(S/K) + (r + stdev ** 2 / 2) * T) / (stdev * np.sqrt(T))

In [33]:
# Valor a pagar por exercer a opção:
def d2(S, K, r, stdev, T):
    return (np.log(S/K) + (r - stdev ** 2 / 2) * T) / (stdev * np.sqrt(T))

Distribuição Normal Cumulativa (cdf):

In [34]:
norm.cdf(0) # 0 como media - esperado que metade dos dados esteja abaixo de 0

0.5

In [35]:
norm.cdf(0.25) # esperado que 60% dos dados estejam abaixo de 0.25

0.5987063256829237

In [36]:
norm.cdf(0.75) # esperado que 77% dos dados estejam abaixo de 0.75

0.7733726476231317

In [37]:
norm.cdf(9) # esperado que 100% dos dados estejam abaixo de 9, pois 9 é considerado como o maior valor da serie

1.0

Definindo a função para a Fórmula de Black-Scholes-Merton:

In [38]:
def BSM(S,K,r,stdev,T):
    return (S * norm.cdf(d1(S,K,r,stdev,T))) - (K * np.exp(-r * T) * norm.cdf(d2(S,K,r,stdev,T)))

Simulando para uma ação:

In [39]:
# Carregando os Dados de uma ação:
ticker = 'PG'
data = pd.DataFrame()
data[ticker] = wb.DataReader(ticker, data_source='yahoo', start='2007-1-1', end='2017-3-21')['Adj Close']

data.tail()

,PG
Date,
2017-03-15,78.513786
2017-03-16,78.548149
2017-03-17,78.170181
2017-03-20,78.359161
2017-03-21,78.333405


In [40]:
# Atribuindo o valor atual da ação a uma variável:
S = data.iloc[-1]

In [41]:
# Calculando os retornos diários da ação:
log_returns = np.log(1 + data.pct_change())
log_returns.tail()

,PG
Date,
2017-03-15,0.004386
2017-03-16,0.000438
2017-03-17,-0.004824
2017-03-20,0.002415
2017-03-21,-0.000329


In [42]:
# Calculando o desvio-padrão anual dos retornos:
stdev = log_returns.std() * 250 ** 0.5
stdev

PG    0.17655
dtype: float64

In [43]:
r = 0.025 # taxa livre de risco - rendimento de um titulo de 10 anos dos EUA
K = 110 # preço de exercício da opção
T = 1 # horizonte de tempo de 1 ano

Executando as Fórmulas:

In [44]:
d1(S,K,r,stdev,T)

PG   -1.693122
dtype: float64

In [45]:
d2(S,K,r,stdev,T)

PG   -1.869672
dtype: float64

In [46]:
BSM(S,K,r,stdev,T)

PG    0.241374
Name: 2017-03-21 00:00:00, dtype: float64

#### Meus Calculos

Parâmentros:
- T - tempo de maturação - prazo para exercício da opção em anos
- S - preço atual de mercado do ativo
- K - preço de exercício da opção
- r - retorno livre de risco (Brasil: SELIC)
- stdev - volatilidade dos retornos do ativo (desvio-padrão ao ano dos retornos)

In [47]:
# Função de d1:
def D1 (T, S, K, r, stdev):
    return (np.log(S / K) + (r + stdev ** 2 / 2) * T) / (stdev * np.sqrt(T))

In [48]:
# Função de d2:
def D2(d1, stdev, T):
    return d1 - (stdev * np.sqrt(T))

In [49]:
# Função de PV(K):
def PVK(K, r, T):
    K * (np.exp(-r * T))

In [50]:
# Preço de Compra:
def call(S, d1, d2, K, T, r):
    return (S * norm.cdf(d1)) - (norm.cdf(d2) * K * np.exp(-r * T))

In [51]:
# Preço de Venda:
def put(S, d1, d2, K, T, r):
    return (norm.cdf(-d2) * K * np.exp(-r * T)) - (norm.cdf(-d1) * S)

In [52]:
d1 = D1(T, S, K, r, stdev)

In [53]:
d2 = D2(d1, stdev, T)

In [54]:
pvk = PVK(K, r, T)

In [57]:
preco_compra = call(S, d1, d2, K, T, r)
preco_compra

PG    0.241374
Name: 2017-03-21 00:00:00, dtype: float64

In [58]:
preco_venda = put(S, d1, d2, K, T, r)
preco_compra

PG    0.241374
Name: 2017-03-21 00:00:00, dtype: float64